<a href="https://colab.research.google.com/github/deckel28/music-mood/blob/master/mood_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import pickle
import h5py
import numpy as np
from numpy import savetxt
from numpy import genfromtxt
from numpy import random
from random import choices
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow import keras
import matplotlib.pyplot as plt
import csv

from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dropout, BatchNormalization
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from sklearn.metrics import multilabel_confusion_matrix

Using TensorFlow backend.


In [3]:
os.chdir('/content/drive/My Drive')
os.listdir()

['GoogleNews-vectors-negative300.bin',
 'feats_train.h5',
 'encoded_train.csv',
 'feats_val.h5',
 'encoded_val.csv',
 'feats_test.h5',
 'encoded_test.csv',
 'model.h5',
 'mood_train.ipynb',
 'mood_pred.ipynb']

In [ ]:
trainY = genfromtxt('encoded_train.csv', delimiter=',')
h5f = h5py.File('./feats_train.h5','r')
trainX = h5f['dataset'][:]
h5f.close()

valY = genfromtxt('encoded_val.csv', delimiter=',')
h5f = h5py.File('./feats_val.h5','r')
valX = h5f['dataset'][:]
h5f.close()

testY = genfromtxt('encoded_test.csv', delimiter=',')
h5f = h5py.File('./feats_test.h5','r')
testX = h5f['dataset'][:]
h5f.close()

In [ ]:
print('trainX-', trainX.shape, '   trainY- ', trainY.shape)
print('valX-', valX.shape, '      valY- ', valY.shape)
print('testX-', testX.shape, '     testY- ', testY.shape)

In [ ]:
trainX0 = np.expand_dims(trainX, axis=-1)
valX0 = np.expand_dims(valX, axis=-1)
testX0 = np.expand_dims(testX, axis=-1)

IN_SHAPE = trainX0.shape[1:]
print(IN_SHAPE)

In [ ]:
print(trainX0.shape, valX0.shape, testX0.shape)
print(trainY.shape, valY.shape, testY.shape)

In [ ]:
model = keras.models.load_model('model.h5')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 20, 211, 64)       640       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 20, 211, 64)       36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 10, 105, 64)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 10, 105, 64)       36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 10, 105, 64)       256       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 10, 105, 64)       36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 52, 64)        

In [ ]:
print(np.sum(data_Y, axis=0))

[183734.  88645.    517.  65082.   9671.  48018.   1675.  21879.]


In [ ]:
def fbeta(y_true, y_pred, beta=2):
  # clip predictions
  y_pred = backend.clip(y_pred, 0, 1)
  # calculate elements
  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
  # calculate precision
  p = tp / (tp + fp + backend.epsilon())
  # calculate recall
  r = tp / (tp + fn + backend.epsilon())
  # calculate fbeta, averaged across each class
  bb = beta ** 2
  fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
  return fbeta_score

In [ ]:
def predict_mood(model, dataX, thresholds):
    predictions = model.predict(dataX)
    y_pred = np.zeros((dataX.shape[0], 8))
    for i in range(dataX.shape[0]):
        for j in range(8):
            y_pred[i][j] = (predictions[i][j] > thresholds[j][1])

    return y_pred, predictions

In [ ]:
def multilabel_cm(trues, preds):
    shape = trues.shape[0]
    cm = np.zeros((8, 8), dtype=int)
    for i in range(shape):
        for j in range(8):
            for k in range(8):
                if trues[i][j] == 1 and preds[i][k] == 1:
                    cm[j][k] = cm[j][k] + 1
    return cm

In [ ]:
def classification_report(trues, preds, thresholds, cm):
    fn = np.zeros((8,))
    fp = np.zeros((8,))
    tp = np.zeros((8,))
    print('CLASSIFICATION REPORT')
    print('---------------------')
    print(' ')

    cm = multilabel_cm(trues, preds)
    fps = np.sum(cm, axis=0)
    fns = np.sum(cm, axis=1)
    occ = np.sum(trues, axis=0)
    report = np.zeros((8, 4))

    for i in range(8):
        tp[i] = cm[i][i]
        fp[i] = fps[i] - cm[i][i]
        fn[i] = fns[i] - cm[i][i]

        precision = tp[i] / (tp[i] + fp[i])
        report[i][0] = precision
        recall = tp[i] / (tp[i] + fn[i])
        report[i][1] = recall
        f1score = 2 * precision * recall / (precision + recall)
        report[i][2] = f1score
        support = occ[i]
        report[i][3] = support

        print(thresholds[i][0])
        print('.....')
        print('Precision:', precision)
        print('Recall:', recall)
        print('F1-Score:', f1score)
        print('Support:', support)
        print('')
        print('')

    return report


In [ ]:
thresholds = [['Happy', 0.8],
                ['Excited', 0.5],
                ['Frantic', 0.5],
                ['Anxious/Sad', 0.3],
                ['Anger', 0.5],
                ['Calm', 0.22],
                ['Tired', 0.5],
                ['Sensual', 0.1]]

In [ ]:
y_pred, y_pred_val = predict_mood(model, testX0, thresholds)
cm = multilabel_cm(testY, y_pred.astype(np.float))
report = classification_report(testY, y_pred.astype(np.float), thresholds, cm)

In [ ]:
tags1 = np.array(tags).reshape((1, 8))
y_pred1 = np.vstack((tags1, y_pred))
print(y_pred1)

In [ ]:
cm1 = np.vstack((tags, cm))
print('-------- Confusion Matrix-----------')
print(cm1)
savetxt('cm.csv', cm1, delimiter=',', fmt='%s')

In [ ]:
report = np.vstack((['Precision', 'Recall', 'F1-Score', 'Support'], report))
x = np.array(thresholds)
names = np.append(['Label'], x[:, 0])
names = names.reshape(9, 1)
report = np.hstack((names, report))
print(report)
savetxt('report.csv', report, delimiter=',', fmt='%s')

In [ ]:
for x in range(2):
    fig, axs = plt.subplots(1, 4)
    for i in range(4):
        axs[i].boxplot(y_pred_val[1:, i + 4 * x].astype(np.float), whis=100)
        axs[i].set_title(thresholds[i + 4 * x][0])
    plt.savefig('train_box_' + str(x) + '.png')


In [ ]:
accuracy = accuracy_score(data_Y[:50], y_pred)
print('Accuracy: %f' % accuracy)

f1 = fbeta(data_Y[:50], y_pred)
print('F1 score: %f' % f1)

hl = hamming_loss(data_Y[:50], y_pred)
print('Hamming Loss: %f' % hl)